In [1]:
import numpy as np

# import Keras items
from keras.models import Model
from keras.layers import Dense,Lambda,Concatenate,Input,Flatten,Reshape
# from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau, TensorBoard
# from keras import backend as K
# from keras.utils import to_categorical,plot_model
# from keras.initializers import constant
# from keras.constraints import MinMaxNor

from itertools import combinations

Using TensorFlow backend.


In [14]:
EPOCHS = 40
BATCH_SIZE = 4
range_min = 0
range_max = 999
train_samples = 1000
test_samples = 200

In [26]:
def func(x):
    out = np.zeros((x.shape[0],3))
    for i in range(x.shape[0]):
        out[i,0] = x[i,8] - x[i,10]
        out[i,1] = x[i,4] - x[i,9]
        out[i,2] = x[i,1] - x[i,6]
        
    return out

In [23]:
def func2(x):
    out = np.zeros((x.shape[0],3))
    for i in range(x.shape[0]):
        out[i,0] = x[i,9] - x[i,13] # bf - ce
        out[i,1] = -(x[i,4] - x[i,12]) # af - cd
        out[i,2] = x[i,3] - x[i,7] # ae - bd
        
    return out

In [24]:
temp = np.random.randint(low=range_min,high=range_max+1,size=(train_samples,6))
x_train = np.zeros(shape=(train_samples,30))
for i in range(train_samples):
    count = 0
    for j in range(6):
        x = temp[i,j]
        for k in range(6):
            if j == k:
                continue
                
            x_train[i,count] = x * temp[i,k]
            count+=1
            
temp = np.random.randint(low=range_min,high=range_max+1,size=(test_samples,6))
x_test = np.zeros(shape=(test_samples,30))
for i in range(test_samples):
    count = 0
    for j in range(6):
        x = temp[i,j]
        for k in range(6):
            if j == k:
                continue
                
            x_test[i,count] = x * temp[i,k]
            count+=1
            
y_train = func2(x_train)
y_test = func2(x_test)

In [28]:
# generate dataset
temp = np.random.randint(low=range_min,high=range_max+1,size=(train_samples,6))
x_train = np.zeros(shape=(train_samples,15))
for i in range(train_samples):
    tuples = list(combinations(temp[i],2))
    tuples = [x[0] * x[1] for x in tuples]
    x_train[i] = np.array(tuples)[:]
    
y_train = func(x_train)

temp = np.random.randint(low=range_min,high=range_max+1,size=(test_samples,6))
x_test = np.zeros(shape=(test_samples,15))
for i in range(test_samples):
    tuples = list(combinations(temp[i],2))
    tuples = [x[0] * x[1] for x in tuples]
    x_test[i] = np.array(tuples)[:]
    
y_test = func(x_test)

In [18]:
def net(inputs):
    input_data = Input(shape=inputs)
    out = Dense(3,use_bias=False)(input_data)
    
    return Model(inputs = input_data,outputs = out)

In [32]:
model = net((30,))
model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 30)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 90        
Total params: 90
Trainable params: 90
Non-trainable params: 0
_________________________________________________________________


In [33]:
out = model.fit(x_train,y_train,
                batch_size=BATCH_SIZE,epochs=36,
                validation_data = (x_test,y_test),verbose=1)

Train on 1000 samples, validate on 200 samples
Epoch 1/36
1000/1000 [==============================] - 0s 306us/step - loss: 165869692370.9440 - acc: 0.3060 - val_loss: 94166455705.6000 - val_acc: 0.5100
Epoch 2/36
1000/1000 [==============================] - 0s 172us/step - loss: 65633634392.0640 - acc: 0.6490 - val_loss: 41174144604.1600 - val_acc: 0.7800
Epoch 3/36
1000/1000 [==============================] - 0s 176us/step - loss: 30557519458.3040 - acc: 0.8190 - val_loss: 20464625684.4800 - val_acc: 0.8550
Epoch 4/36
1000/1000 [==============================] - ETA: 0s - loss: 17664816668.4444 - acc: 0.876 - 0s 176us/step - loss: 17574055582.7200 - acc: 0.8770 - val_loss: 13141348428.8000 - val_acc: 0.8800
Epoch 5/36
1000/1000 [==============================] - 0s 240us/step - loss: 12761416636.9280 - acc: 0.8980 - val_loss: 10295615695.3600 - val_acc: 0.8900
Epoch 6/36
1000/1000 [==============================] - 0s 265us/step - loss: 10283826629.6320 - acc: 0.9050 - val_loss: 842

In [21]:
model.get_weights()[0]

array([[-0.00522966, -0.09576197, -0.00224456],
       [ 0.06236473,  0.08222419,  0.00779453],
       [-0.01317273,  0.15182112,  0.03774474],
       [-0.08200622, -0.28984204,  0.47226918],
       [ 0.19897032,  0.33277747,  0.2805949 ],
       [ 0.0052297 ,  0.09576203,  0.00224456],
       [-0.09008911, -0.20640856,  0.08586637],
       [-0.1325207 , -0.18708298, -0.6218502 ],
       [-0.22400635,  0.31712112,  0.10735036],
       [ 0.5118019 , -0.15424497,  0.19211707],
       [-0.06236456, -0.08222401, -0.00779453],
       [ 0.09008916,  0.20640852, -0.08586638],
       [-0.20106779, -0.72781014,  0.03681654],
       [-0.5456294 ,  0.16088825, -0.39139682],
       [ 0.11445398, -0.16651456,  0.20971715],
       [ 0.01317275, -0.15182103, -0.03774468],
       [ 0.13252082,  0.18708283, -0.3781499 ],
       [ 0.20106776, -0.27218997, -0.03681654],
       [ 0.06622057,  0.15154195,  0.22442633],
       [-0.24209519,  0.3004822 , -0.03866903],
       [ 0.08200616,  0.2898421 ,  0.527

In [34]:
model.save_weights('regression_test.h5')